<a href="https://colab.research.google.com/github/aleighbrown/pangolin_run_cryptic_biology/blob/main/PangolinColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Pangolin on Colab

*   List item
*   List item



0. By default, Colab will not use a GPU. To use a GPU, choose: Runtime -> Change runtime type -> GPU.  

1. Install Pangolin and dependencies


2. Upload input files. For details on ways to do this (for example, through Google Drive), see: https://colab.research.google.com/notebooks/io.ipynb


3. Run Pangolin

In [5]:
import pandas as pd

In [10]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving whatever.csv to whatever.csv
User uploaded file "whatever.csv" with length 1878227 bytes


In [11]:
df = pd.read_csv('whatever.csv')
df

,sequence,ID_exon,type,gene,paste_into_igv_junction
0,ACATGGTAACTGGACGTTTCACTTTTTGAGGAGCTGCGAGACTGTT...,3,acceptor,ABCD1,chrX:153740237-153740305
1,CTTTGTATTCTAGCCACCTAGTGGGTGTGAGGCAGTATCTCTTGGT...,3,donor,ABCD1,chrX:153740439-153740574
2,CTTCCGGACGCAATTTTGAGGAGGTCGGAGCGGAAGGACGTCGTGG...,7,acceptor,ACAD10,chr12:111690553-111691059
3,TACTGGGCTTGGAGGCCGGAGCTGGGGAGGCCCAGCTCTTCCTACA...,9,acceptor,ACOT11,chr1:54599415-54600963
4,GATGCCAAGGTGGGAGAATCACTTGAGCCCAGGAGTTCGAGACCAG...,11,acceptor,ACSF2,chr17:50461686-50461820
...,...,...,...,...,...
182,AGGCTATACCATCTAGGTTGTGTAAGTATACTCCATGATGTTCATA...,509,donor,ZNF570,chr19:37469655-37470304
183,ACAAAACAAACACAAGAAAAGCCAGTCTGACAACACTGGTTTATTC...,510,acceptor,ZNF608,chr5:124746283-124748533
184,AACTCCCTCCCTGGATCTCAAAGCTCTCCTAAAGGCACTTAATTTG...,513,acceptor,ZNF681,chr19:23747101-23754823
185,TCCTCTCTGCTTATATCTAGCATGCCTGTTTTGGGTGGTCCCTGGG...,515,acceptor,ZNF700,chr19:11925273-11947172


In [12]:
import numpy as np
import pandas as pd
import torch
from pkg_resources import resource_filename
from pangolin.model import * # NOTE: L, W, and AR must be defined within pangolin.model or manually set here.

# --- Configuration ---
INPUT_CSV = 'whatever.csv'
OUTPUT_CSV = 'pangolin_predictions.csv'

# Define the models we want to run (4: Brain P(splice), 5: Brain usage)
model_nums = [4, 5]

# Map model number to output layer index
INDEX_MAP = {0:1, 1:2, 2:4, 3:5, 4:7, 5:8, 6:10, 7:11}

# --- Data Preparation Functions ---

IN_MAP = np.asarray([[0, 0, 0, 0],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def one_hot_encode(seq, strand):
    """Converts a DNA sequence to a one-hot encoded matrix."""
    seq = seq.upper().replace('A', '1').replace('C', '2')
    seq = seq.replace('G', '3').replace('T', '4').replace('N', '0')

    if strand == '+':
        seq_array = np.asarray(list(map(int, list(seq))))
    elif strand == '-':
        seq_array = np.asarray(list(map(int, list(seq[::-1]))))
        seq_array = (5 - seq_array) % 5  # Reverse complement
    else:
        raise ValueError("Strand must be '+' or '-'.")

    return IN_MAP[seq_array.astype('int8')]

def load_models(m_nums):
    """Loads a set of 5 ensemble models for each specified model number."""
    all_models = {}
    for i in m_nums:
        models_for_num = []
        for j in range(1, 6): # Load 5 ensemble models
            model = Pangolin(L, W, AR)
            try:
                if torch.cuda.is_available():
                    model.cuda()
                    weights = torch.load(resource_filename("pangolin", f"models/final.{j}.{i}.3"))
                else:
                    weights = torch.load(resource_filename("pangolin", f"models/final.{j}.{i}.3"),
                                         map_location=torch.device('cpu'))
                model.load_state_dict(weights)
                model.eval()
                models_for_num.append(model)
            except Exception as e:
                print(f"Warning: Could not load model {j} for model_num {i}: {e}")
                continue
        all_models[i] = models_for_num
    return all_models

# --- Main Execution ---

# 1. Read the input CSV
try:
    df = pd.read_csv(INPUT_CSV)
except FileNotFoundError:
    print(f"Error: Input file '{INPUT_CSV}' not found.")
    print("Please ensure your CSV exists and has the required columns.")
    exit()

# Check for required columns
REQUIRED_COLS = ['gene', 'sequence', 'type', 'paste_into_igv_junction', 'ID_exon']
if not all(col in df.columns for col in REQUIRED_COLS):
    missing = [col for col in REQUIRED_COLS if col not in df.columns]
    print(f"Error: Input CSV is missing the following required columns: {', '.join(missing)}")
    exit()

# Add a default strand assumption if not present (assuming '+')
if 'strand' not in df.columns:
    df['strand'] = '+'
print(f"Processing {len(df)} sequences...")

# 2. Load all necessary models
all_models = load_models(model_nums)

# 3. Prepare the output DataFrame structure
results = []

# 4. Iterate through each row (sequence) in the input DataFrame
for index, row in df.iterrows():
    seq_data = row['sequence']
    strand_data = row['strand']

    if len(seq_data) != 10001:
        print(f"Warning: Row {index} ({row['gene']}) has sequence length {len(seq_data)}, expected 10001. Skipping.")
        continue

    # 4a. One-hot encode the sequence
    try:
        seq_oh = one_hot_encode(seq_data, strand_data).T
        seq_tensor = torch.from_numpy(np.expand_dims(seq_oh, axis=0)).float()

        if torch.cuda.is_available():
            seq_tensor = seq_tensor.to(torch.device("cuda"))

    except Exception as e:
        print(f"Skipping row {index} ({row['gene']}) due to encoding error: {e}")
        continue

    # 4b. Collect all model predictions and preserve metadata
    prediction_row = {
        'gene': row['gene'],
        'type': row['type'],
        'paste_into_igv_junction': row['paste_into_igv_junction'],  # PRESERVED
        'ID_exon': row['ID_exon']                                   # PRESERVED
    }

    for model_num in model_nums:
        scores = []
        # Average across the 5 ensemble models
        for model in all_models[model_num]:
            with torch.no_grad():
                score_output = model(seq_tensor)[0][INDEX_MAP[model_num], :].cpu().numpy()
                scores.append(score_output)

        mean_score_array = np.mean(scores, axis=0)

        # Extract the single score for the center position (as input is 10001 bp)
        col_name = 'P(splice)' if model_num == 4 else 'usage'
        prediction_row[col_name] = mean_score_array[0]

    results.append(prediction_row)

# 5. Create the final output DataFrame
output_df = pd.DataFrame(results)

# 6. Save the output
output_df.to_csv(OUTPUT_CSV, index=False)
print(f"\nSuccessfully wrote predictions to '{OUTPUT_CSV}'.")
print("\nOutput DataFrame structure:")
print(output_df[['gene', 'P(splice)', 'usage', 'type', 'paste_into_igv_junction', 'ID_exon']].head())

/tmp/ipython-input-773582.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename


Processing 187 sequences...

Successfully wrote predictions to 'pangolin_predictions.csv'.

Output DataFrame structure:
     gene  P(splice)     usage      type    paste_into_igv_junction  ID_exon
0   ABCD1   0.002025  0.000042  acceptor   chrX:153740237-153740305        3
1   ABCD1   0.016729  0.000068     donor   chrX:153740439-153740574        3
2  ACAD10   0.006058  0.000673  acceptor  chr12:111690553-111691059        7
3  ACOT11   0.428372  0.057887  acceptor     chr1:54599415-54600963        9
4   ACSF2   0.191023  0.016174  acceptor    chr17:50461686-50461820       11


In [13]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download(OUTPUT_CSV)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>